In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex
import datetime
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore')

# Show 2 decimals
pd.set_option('display.float_format', lambda x: '%.0f' % x)
#pd.options.display.float_format = '{:,.2f}'.format

#Butun kolon ve satirlari gormek icin
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set plot background
sns.set_theme(style="whitegrid")

# Euro sign €
#print ("%s"%(u"\N{euro sign}"))

In [2]:
auto = pd.read_csv("/Users/macbook/Desktop/csv_dosyalari/autoscout_data_2000.csv")
df = auto.copy()
df.head(2)

,location,url,make,model,short_description,price,mileage,gearbox,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty,Mileage,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Power,Gearbox,engine_size,gears,cylinders,empty_weight,Fuel_type,fuel_consumption,co2_emissions,emission_class,emissions_sticker,\nComfort & Convenience\n,\nEntertainment & Media\n,\nSafety & Security\n,extras,\ncolour,\nmanufacturer_color,\npaint,upholstery_colour,upholstery
0,"LOON OP ZAND, NL",https://www.autoscout24.com/offers/audi-a3-2-0...,"\n,Audi ,\n","\n,A3,\n",2.0 FSI Ambition Pro Line,"[['\n', '€ 4,450.-', '\n']]","['', ['191,066 km']]","\n,Automatic,\n",08/2003,"\n,Gasoline,\n","110 kW (150 hp),","['\n', ['Dealer\n']]","\n,Compact,\n","\n,Used,\n","\n,Front,\n",\n5,\n3,"Country version,None",\nSJ-112-R,"\n,6 months,\n","[['\n'], ['']]",08/2003,None,05/2022,None,"\n,Yes,\n","\n,None,\n","[['\n'], ['None']]","[['\n'], ['None']]","\n,110 kW (150 hp),\n","\n,Automatic,\n","\n,1,984 cc,\n","\n,None,\n","\n,4,\n","\n,1,280 kg,\n","\n,Super 95,\n","\n,None,\n","['\n', [' ', ' ', ' '], '\n']","\n,Euro 4,\n","\n,None,\n","[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],"\n4, Grey",None,"\npaint, Metallic","\nupholstery, Black","\n8, Cloth"
1,"HEERLEN, NL",https://www.autoscout24.com/offers/audi-a3-spo...,"\n,Audi ,\n","\n,A3,\n",Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,"[['\n', '€ 12,950.-', '\n']]","['', ['94,758 km']]","\n,Automatic,\n",10/2013,"\n,Gasoline,\n","92 kW (125 hp),","['\n', ['Dealer\n']]","\n,Compact,\n","\n,Used,\n","\n,Front,\n",\n5,\n5,"Country version,None",\n809725,"\n,None,\n","[['\n'], ['']]",10/2013,None,None,None,"\n,Yes,\n","\n,None,\n","[['\n'], ['None']]","[['\n'], ['None']]","\n,92 kW (125 hp),\n","\n,Automatic,\n","\n,1,395 cc,\n","\n,None,\n","\n,4,\n","\n,1,200 kg,\n","\n,Gasoline,\n","['\n', ['4.9 l/100 km (comb.)', ' ', ' '], '\n']","\n,None,\n","\n,None,\n","\n,None,\n","[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]","\n4, White",None,None,"\nupholstery, Black","\n8, Cloth"


In [3]:
# Drop Duplicated Columns and Url column
df.drop(['url', 'Mileage', 'Power', 'gearbox'], axis=1, inplace= True)

df.columns

Index(['location', 'make', 'model', 'short_description', 'price', 'mileage',
       'registration', 'fuel_type', 'power', 'seller', 'body_type', 'type',
       'drivetrain', 'seats', 'doors', 'country_version', 'offer_number',
       'warranty', 'first_registration', 'production_date',
       'general_inspection', 'last_service', 'full_service_history',
       'non_smoker_vehicle', 'previous_owner', 'last_timing_belt_change',
       'Gearbox', 'engine_size', 'gears', 'cylinders', 'empty_weight',
       'Fuel_type', 'fuel_consumption', 'co2_emissions', 'emission_class',
       'emissions_sticker', '\nComfort & Convenience\n',
       '\nEntertainment & Media\n', '\nSafety & Security\n', 'extras',
       '\ncolour', '\nmanufacturer_color', '\npaint', ' upholstery_colour ',
       ' upholstery '],
      dtype='object')

In [4]:
# 1 - Rename and Reassign Columns (move unwanted characters from column names)
df.columns = ['location', 'make', 'model', 'short_description', 'price','mileage_km', 'registration', 'fuel_type', 'power', 'seller',
       'body_type', 'type', 'drivetrain', 'seats', 'doors', 'country_version','offer_number', 'warranty_months',  'first_registration',
       'production_date', 'general_inspection', 'last_service','full_service_history', 'non_smoker_vehicle', 'previous_owner',
       'last_timing_belt_change', 'Gearbox', 'engine_size_cc', 'gears','cylinders', 'empty_weight_kg', 'Fuel_type', 'fuel_consumption',
       'co2_emissions_g_km_comb', 'emission_class', 'emissions_sticker','Comfort & Convenience', 'Entertainment & Media',
       'Safety & Security', 'extras', 'colour', 'manufacturer_color','paint', 'upholstery_colour', 'upholstery']

In [5]:
# 2 - Removing unwanted Chracters from data
# Except these colums ====> location, short_description, registration, first_registration, general_inspection, Comfort & Convenience, Entertainment & Media, Safety & Security, extras
clmns = ['make', 'model', 'price','mileage_km', 'fuel_type', 'power', 'seller',
       'body_type', 'type', 'drivetrain', 'seats', 'doors', 'country_version','offer_number', 'warranty_months',
       'production_date', 'last_service','full_service_history', 'non_smoker_vehicle', 'previous_owner',
       'last_timing_belt_change', 'Gearbox', 'engine_size_cc', 'gears','cylinders', 'empty_weight_kg', 'Fuel_type', 'fuel_consumption',
       'co2_emissions_g_km_comb', 'emission_class', 'emissions_sticker',
       'colour', 'manufacturer_color','paint', 'upholstery_colour', 'upholstery']

for column in clmns:
    df[column] = df[column].str.replace(r'\W', '')
    
df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,"LOON OP ZAND, NL",Audi,A3,2.0 FSI Ambition Pro Line,n4450n,191066km,08/2003,Gasoline,110kW150hp,nDealern,Compact,Used,Front,5,3,CountryversionNone,SJ112R,6months,08/2003,None,05/2022,None,Yes,None,nNone,nNone,Automatic,1984cc,None,4,1280kg,Super95,None,nn,Euro4,None,"[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],4Grey,None,paintMetallic,upholsteryBlack,8Cloth
1,"HEERLEN, NL",Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,n12950n,94758km,10/2013,Gasoline,92kW125hp,nDealern,Compact,Used,Front,5,5,CountryversionNone,809725,None,10/2013,None,None,None,Yes,None,nNone,nNone,Automatic,1395cc,None,4,1200kg,Gasoline,n49l100kmcombn,None,None,None,"[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]",4White,None,None,upholsteryBlack,8Cloth


In [6]:
# 3 - Delete NL from Location
df.location = df['location'].str.replace(', NL', '').str.upper()
df.location = df['location'].str.replace('.', '').str.replace('5', '').str.replace('6', '').str.replace(r'\+318429897', '')
df.location = df.location.replace('', np.NaN)

# Delete n from columns
L01 = ['seller', 'price', 'previous_owner', 'last_timing_belt_change', 'fuel_consumption', 'co2_emissions_g_km_comb']
for i in L01:
    df[i] = df[i].str.replace('n', '')

# Model nNonen to np.NaN
df.model = df.model.str.replace('nNonen', '').replace('', np.NaN)

# Delete [ ] from L02    
L02 = ['Comfort & Convenience', 'Entertainment & Media', 'Safety & Security', 'extras']
for i in L02:
    df[i] = df[i].str.replace('[', '')
    df[i] = df[i].str.replace(']', '')
    
# delete 4 from colour
df['colour'] = df['colour'].str.replace('4', '')

# Delete paint from paint column
df['paint'] = df['paint'].str.replace('paint', '')

# Delete upholstery from upholstery_colour
df['upholstery_colour'] = df['upholstery_colour'].str.replace('upholstery', '')

# Delete 8 from upholstery
df['upholstery'] = df['upholstery'].str.replace('8', '')

df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066km,08/2003,Gasoline,110kW150hp,Dealer,Compact,Used,Front,5,3,CountryversionNone,SJ112R,6months,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984cc,None,4,1280kg,Super95,Noe,,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth
1,HEERLEN,Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,12950,94758km,10/2013,Gasoline,92kW125hp,Dealer,Compact,Used,Front,5,5,CountryversionNone,809725,None,10/2013,None,None,None,Yes,None,Noe,Noe,Automatic,1395cc,None,4,1200kg,Gasoline,49l100kmcomb,Noe,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth


In [7]:
df['mileage_km'] = df['mileage_km'].str.replace('km', '')
df['warranty_months'] = df['warranty_months'].str.replace('months', '')
df['engine_size_cc'] = df['engine_size_cc'].str.replace('cc', '')
df['empty_weight_kg'] = df['empty_weight_kg'].str.replace('kg', '')
df['country_version'] = df['country_version'].str.replace('Countryversion', '')
df['co2_emissions_g_km_comb'] = df['co2_emissions_g_km_comb'].str.replace('gkmcomb', '')
df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066,08/2003,Gasoline,110kW150hp,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth
1,HEERLEN,Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,12950,94758,10/2013,Gasoline,92kW125hp,Dealer,Compact,Used,Front,5,5,None,809725,None,10/2013,None,None,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,Noe,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth


In [8]:
df1 = df.copy()

L03 = ['price', 'mileage_km','engine_size_cc', 'empty_weight_kg', 'seats', 'doors', 'warranty_months', 'cylinders', 'co2_emissions_g_km_comb']
for i in L03:
    df[i] = df[i].replace(['None', 'Noe', 'Yes', 'Automatic', 'Manual', 'Semiautomatic'], np.NaN)
    df[i] = df[i].replace('', np.NaN)
    df[i] = df[i].astype(float)

df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,69534,24369,468620,1,8945,17450,28935,123456789
mileage_km,70951,107148,88475,1,30194,95615,163794,2215714
seats,65562,5,1,1,5,5,5,14
doors,69790,5,1,1,4,5,5,6
warranty_months,24004,8,9,1,6,6,6,84
engine_size_cc,65265,1771,702,0,1390,1598,1995,65535
cylinders,64809,4,1,1,4,4,4,12
empty_weight_kg,69500,1433,321,1,1234,1382,1591,4200
co2_emissions_g_km_comb,52190,129,68,0,105,133,164,5900


In [9]:
# Find the rows which have no price
price_null_tf = (df['price'].isnull())
price_null = (df[price_null_tf])
price_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1570 entries, 54 to 71065
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   location                 1570 non-null   object 
 1   make                     1570 non-null   object 
 2   model                    1570 non-null   object 
 3   short_description        1567 non-null   object 
 4   price                    0 non-null      float64
 5   mileage_km               1570 non-null   float64
 6   registration             1570 non-null   object 
 7   fuel_type                1570 non-null   object 
 8   power                    1570 non-null   object 
 9   seller                   1570 non-null   object 
 10  body_type                1570 non-null   object 
 11  type                     1570 non-null   object 
 12  drivetrain               1570 non-null   object 
 13  seats                    1471 non-null   float64
 14  doors                 

In [10]:
L04 = ['power_kw', 'power_hp']
df[L04] = df.power.str.split('kW', expand= True)   
df['power_hp'] = df['power_hp'].str.replace('hp', '').replace('None', np.NaN) 
df['power_kw'] = df['power_kw'].str.replace('kg', '').replace('None', np.NaN)     
df[L04] = df[L04].astype(float)
df = df.drop(['power'], axis=1)
df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066,08/2003,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150
1,HEERLEN,Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,12950,94758,10/2013,Gasoline,Dealer,Compact,Used,Front,5,5,None,809725,NaN,10/2013,None,None,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,NaN,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth,92,125


In [11]:
L05 = ['registration', 'first_registration', 'general_inspection']
for i in L05:
    df[i] = pd.to_datetime(df[i], format= '%m/%Y', errors= 'coerce')

df['production_date'] = pd.to_datetime(df['production_date'], format= '%Y', errors= 'coerce')

# Creating new Features df_dates
df_dates = pd.DataFrame({'registration':  df.registration,
                         'first_registration' : df.first_registration,
                         'production_date' : df.production_date,
                         'general_inspection' : df.general_inspection,
                        'last_timing_belt_change' : df.last_timing_belt_change})

L05 = ['registration', 'first_registration', 'general_inspection']
for i in L05:
    df_dates[i] = pd.to_datetime(df_dates[i], format= '%m/%Y', errors= 'coerce')

df_dates['production_date'] = pd.to_datetime(df_dates['production_date'], format= '%Y', errors= 'coerce')
df_dates['last_timing_belt_change'] = pd.to_datetime(df_dates['last_timing_belt_change'], format= '%m%Y', errors= 'coerce')
df_dates.head()

,registration,first_registration,production_date,general_inspection,last_timing_belt_change
0,2003-08-01,2003-08-01,NaT,2022-05-01,NaT
1,2013-10-01,2013-10-01,NaT,NaT,NaT
2,2013-05-01,2013-05-01,NaT,2022-03-01,NaT
3,2015-06-01,2015-06-01,2015-01-01,NaT,NaT
4,2015-10-01,2015-10-01,NaT,NaT,NaT


In [12]:
df2 = df.copy()

In [13]:
# Add year Column !!!!!!
pd.options.display.float_format = '{:.0f}'.format 
df.insert(4, 'year', df.registration.dt.year, True )
df.insert(3, 'make_model', (df['make'].astype(str) +"-"+ df["model"].astype(str)), True)
df.head(2)

,location,make,model,make_model,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
0,LOON OP ZAND,Audi,A3,Audi-A3,2.0 FSI Ambition Pro Line,2003,4450,191066,2003-08-01,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,2003-08-01,NaT,2022-05-01,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150
1,HEERLEN,Audi,A3,Audi-A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,2013,12950,94758,2013-10-01,Gasoline,Dealer,Compact,Used,Front,5,5,None,809725,NaN,2013-10-01,NaT,NaT,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,NaN,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth,92,125


# Wrong Values to NaN

In [14]:
# Year 2025 and 2028 to np.nan
df['year'] = df['year'].replace([2025, 2028], np.NaN)
df.insert(4, 'age', (2022 - df.year), True)
df.head(2)

,location,make,model,make_model,age,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
0,LOON OP ZAND,Audi,A3,Audi-A3,19,2.0 FSI Ambition Pro Line,2003,4450,191066,2003-08-01,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,2003-08-01,NaT,2022-05-01,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150
1,HEERLEN,Audi,A3,Audi-A3,9,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,2013,12950,94758,2013-10-01,Gasoline,Dealer,Compact,Used,Front,5,5,None,809725,NaN,2013-10-01,NaT,NaT,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,NaN,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth,92,125


In [15]:
german = ['Audi', 'MercedesBenz', 'BMW', 'Volkswagen', 'Opel', 'Skoda']
japan = ['Toyota', 'Honda', 'Mazda']
sweden = ['Volvo']
france = ['Renault', 'Peugeot', 'Citroen']
america = ['Ford', 'Chevrolet', 'Tesla']
south_korea = ['Kia', 'Hyundai']
italy = ['Fiat']
romanian = ['Dacia']

conditions = [ 
    (df.make.isin(german)),
    (df.make.isin(japan)),
    (df.make.isin(sweden)),
    (df.make.isin(france)),
    (df.make.isin(america)),
    (df.make.isin(south_korea)),
    (df.make.isin(italy)),
    (df.make.isin(romanian)),
]

values = ['German', 'Japan', 'Sweden', 'France', 'America', 'South_Korea', 'Italy', 'Romanian']

df['make_country'] = np.select(conditions, values)
df.tail(1)

,location,make,model,make_model,age,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp,make_country
71103,SOESTERBERG,Volvo,S60,Volvo-S60,21,"2.4 Airco, Cruise Control, Stuurbekrachtiging",2001,944,404758,2001-09-01,LPG,Dealer,Sedan,Used,Front,5,4,None,26HNLS,NaN,2001-09-01,NaT,NaT,None,None,None,Noe,Noe,Manual,2435,5,5,1488,LiquidpetroleumgasLPG,84l100kmcomb116l100kmcity65l100kmcoutry,207,Euro3,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'On-board computer', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,None,Partleather,104,141,Sweden


In [16]:
df.price.isnull().sum()

1570

# PRICE

In [17]:
# Wrong Price to NaN

# high price to NaN
#index_list = [1406, 22706, 23072, 66537, 71027]
#df.loc[index_list, 'price'] = np.NaN

# low price to NaN



df.price.isnull().sum()

1570

In [18]:
df.price.max()

123456789.0

# Outliers

In [19]:
df.loc[df.price >250000, ['price', 'make', 'model']]

,price,make,model
1245,306894,Audi,R8
1406,123456789,Audi,TT
7413,279950,BMW,Z8
19883,250845,MercedesBenz,AMGGT
20417,290500,MercedesBenz,G63AMG
21805,1250000,MercedesBenz,AMGGT
22706,999999,MercedesBenz,G400
23072,999999,MercedesBenz,G400
66537,999999,Volvo,C70
71027,999999,Volvo,C70


In [20]:
list_short = ['make', 'model', 'price', 'year','age','mileage_km']
df.loc[(df.price <=500) & (df.mileage_km <=100000) , list_short]

,make,model,price,year,age,mileage_km
14588,Ford,KaKa,499,2001,21,31949
17729,Ford,Transit,275,2017,5,77521
50269,Kia,Picanto,239,2022,0,10
52116,Kia,eNiro,464,2022,0,10
52122,Kia,eNiro,484,2022,0,10


In [21]:
df.loc[(df.price <=5000) & (df.mileage_km <=10000) , list_short]

,make,model,price,year,age,mileage_km
9896,BMW,116,4500,2006,16,186
23050,MercedesBenz,Vito,4700,2007,15,250
23882,MercedesBenz,300,2700,2000,22,3000
32155,Volkswagen,Golf,4900,2010,12,1
32160,Volkswagen,Polo,4900,2010,12,1
50269,Kia,Picanto,239,2022,0,10
52113,Kia,EV6,579,2021,1,10
52116,Kia,eNiro,464,2022,0,10
52118,Kia,eNiro,539,2022,0,10
52122,Kia,eNiro,484,2022,0,10


In [22]:
df.loc[(df.make_model == 'Kia-eNiro') & (df.year == 2022)].describe().T

,count,mean,std,min,25%,50%,75%,max
age,13,0,0,0,0,0,0,0
year,13,2022,0,2022,2022,2022,2022,2022
price,13,33938,19182,464,40790,42017,45457,47252
mileage_km,12,13,3,10,10,15,15,15
seats,10,5,0,5,5,5,5,5
doors,13,5,0,5,5,5,5,5
warranty_months,1,84,NaN,84,84,84,84,84
engine_size_cc,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
empty_weight_kg,13,1712,0,1712,1712,1712,1712,1712


In [23]:
df.loc[(df.make_model == 'Kia-eNiro') & (df.year == 2022), list_short]

,make,model,price,year,age,mileage_km
51841,Kia,eNiro,45457,2022,0,15
51842,Kia,eNiro,45457,2022,0,15
51843,Kia,eNiro,45457,2022,0,15
51902,Kia,eNiro,42017,2022,0,15
51905,Kia,eNiro,42017,2022,0,15
51907,Kia,eNiro,42017,2022,0,15
51912,Kia,eNiro,42017,2022,0,15
52026,Kia,eNiro,47252,2022,0,10
52030,Kia,eNiro,47230,2022,0,10
52091,Kia,eNiro,40790,2022,0,NaN


In [25]:
df.to_csv('auto_02.csv')

In [26]:
df.duplicated().sum()

2012

In [28]:
df.loc[df.duplicated(), :].head()

,location,make,model,make_model,age,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp,make_country
638,WOMMELS,Audi,A3,Audi-A3,16,1.6 FSI Ambiente,2006,2750,289500,2006-11-01,Gasoline,Privateseller,Compact,Used,None,5,4,None,None,NaN,2006-11-01,NaT,2022-06-01,None,Yes,None,Noe,Noe,Manual,1598,6,4,1240,Gasoline,67l100kmcomb89l100kmcity54l100kmcoutry,161,Euro4,None,"'Air conditioning', 'Automatic climate control...","'Hands-free equipment', 'On-board computer', '...","'Central door lock', 'Central door lock with r...","'Alloy wheels (16"")', 'Automatically dimming i...",Blue,None,None,None,None,85,116,German
1199,NAARDEN,Audi,TTS,Audi-TTS,5,Roadster 2.0 TFSI quattro Hoofdruimteverwarmin...,2017,49900,5076,2017-10-01,Gasoline,Dealer,Convertible,Used,4WD,2,2,None,PV399Z,6,2017-10-01,NaT,NaT,None,Yes,None,Noe,Noe,Automatic,1984,6,4,1445,Super95,69l100kmcomb84l100kmcity6l100kmcoutry,159,Euro6,None,"'Air conditioning', 'Cruise control', 'Electri...","'Digital cockpit', 'Hands-free equipment', 'Ra...","'ABS', 'Alarm system', 'Bi-Xenon headlights', ...","'Alloy wheels (18"")', 'Automatically dimming i...",Black,None,Metallic,Black,Fullleather,228,310,German
1201,HENGELO,Audi,A4,Audi-A4,17,cabrio Youngtimer AUDI A4 CABRIOLET; 120 KW AUT,2005,4950,236880,2005-11-01,Gasoline,Dealer,Convertible,Used,None,4,2,None,73SRVL,NaN,2005-11-01,NaT,2022-12-01,None,Yes,None,Noe,Noe,Automatic,1781,None,4,1555,Gasoline,Noe,NaN,None,None,"'Air conditioning', 'Leather steering wheel', ...","'CD player', 'Radio'","'ABS', 'Central door lock', 'Central door lock...",'Alloy wheels',Black,None,Metallic,Black,Fullleather,120,163,German
1206,AMMERZODEN,Audi,A3,Audi-A3,13,Cabriolet 1.8 TFSI Attraction / AIRCO / STOELV...,2009,7499,222849,2009-03-01,Gasoline,Dealer,Convertible,Used,Front,4,2,None,84HXN5,NaN,2009-03-01,NaT,NaT,None,None,None,Noe,Noe,Manual,1798,None,4,1400,Super95,73l100kmcomb10l100kmcity57l100kmcoutry,174,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'On-board computer', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...","'Alloy wheels (17"")', 'Trailer hitch'",Grey,None,None,None,None,118,160,German
1207,BLARICUM,Audi,Cabriolet,Audi-Cabriolet,18,Cabriolet,2004,5000,235000,2004-01-01,Gasoline,Privateseller,Convertible,Used,Front,4,2,None,None,NaN,2004-01-01,NaT,2022-05-01,None,Yes,Yes,Noe,Noe,Automatic,1781,None,4,1690,Gasoline,Noe,NaN,nn,None,"'Air conditioning', 'Armrest', 'Auxiliary heat...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock with...","'Ski bag', 'Spare tyre', 'Steel wheels', 'Summ...",Blue,None,None,Grey,Fullleather,125,170,German


In [34]:
(df.loc[df.make_model == 'Audi-A3', :]).loc[df.duplicated(subset= df.columns), :]

,location,make,model,make_model,age,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp,make_country
638,WOMMELS,Audi,A3,Audi-A3,16,1.6 FSI Ambiente,2006,2750,289500,2006-11-01,Gasoline,Privateseller,Compact,Used,None,5,4,None,None,NaN,2006-11-01,NaT,2022-06-01,None,Yes,None,Noe,Noe,Manual,1598,6,4,1240,Gasoline,67l100kmcomb89l100kmcity54l100kmcoutry,161,Euro4,None,"'Air conditioning', 'Automatic climate control...","'Hands-free equipment', 'On-board computer', '...","'Central door lock', 'Central door lock with r...","'Alloy wheels (16"")', 'Automatically dimming i...",Blue,None,None,None,None,85,116,German
1206,AMMERZODEN,Audi,A3,Audi-A3,13,Cabriolet 1.8 TFSI Attraction / AIRCO / STOELV...,2009,7499,222849,2009-03-01,Gasoline,Dealer,Convertible,Used,Front,4,2,None,84HXN5,NaN,2009-03-01,NaT,NaT,None,None,None,Noe,Noe,Manual,1798,None,4,1400,Super95,73l100kmcomb10l100kmcity57l100kmcoutry,174,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'On-board computer', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...","'Alloy wheels (17"")', 'Trailer hitch'",Grey,None,None,None,None,118,160,German
1208,REUVER,Audi,A3,Audi-A3,12,"Cabriolet 1.2 TFSI Ambition Stoelverwarming, T...",2010,7999,144782,2010-05-01,Gasoline,Dealer,Convertible,Used,Front,4,2,None,797177,NaN,2010-05-01,NaT,NaT,None,None,None,Noe,Noe,Manual,1197,6,4,1360,Gasoline,57l100kmcomb,NaN,None,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'On-board computer', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...","'Alloy wheels (17"")', 'Sport seats', 'Sport su...",Silver,None,Metallic,Black,None,77,105,German
1214,WOERDEN,Audi,A3,Audi-A3,10,Cabriolet 1.2 TFSI Ambition Pro Line S,2012,8500,231000,2012-02-01,Gasoline,Privateseller,Convertible,Used,Front,4,2,None,None,NaN,2012-02-01,NaT,2022-02-01,None,Yes,Yes,Noe,Noe,Manual,1197,6,4,1335,Gasoline,57l100kmcomb7l100kmcity5l100kmcoutry,132,Euro5,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'Apple CarPlay', 'Bluetooth', 'MP3', 'Radio', ...","'ABS', 'Central door lock with remote control'...","'Alloy wheels (17"")', 'Sport seats', 'Sport su...",Grey,None,Metallic,Black,Partleather,77,105,German
2694,AMSTERDAM,Audi,A3,Audi-A3,14,1.4 TFSI Sportback Ambition,2008,5500,210710,2008-07-01,Gasoline,Privateseller,Stationwagon,Used,None,5,5,None,None,NaN,2008-07-01,NaT,NaT,None,None,None,Noe,Noe,Manual,1390,6,4,1295,Gasoline,58l100kmcomb73l100kmcity49l100kmcoutry,134,Euro5,None,"'Electrical side mirrors', 'Leather steering w...","'Bluetooth', 'CD player', 'On-board computer',...","'ABS', 'Central door lock', 'Driver-side airba...","'Alloy wheels', 'Sport seats', 'Sport suspension'",Blue,None,None,None,None,92,125,German
3334,HOLLANDS KROON,Audi,A3,Audi-A3,7,1.4 TFSI cylinder on demand ultra Limousine Am...,2015,17900,155000,2015-09-01,Gasoline,Privateseller,Sedan,Used,None,5,4,None,None,NaN,2015-09-01,NaT,NaT,None,None,None,Noe,Noe,Manual,1395,6,4,1315,Gasoline,48l100kmcomb59l100kmcity41l100kmcoutry,110,Euro6,None,"'Air conditioning', 'Electrical side mirrors',...","'CD player', 'MP3', 'On-board computer', 'Radio'","'ABS', 'Central door lock with remote control'...","'Alloy wheels', 'Electronic parking brake', 'S...",Grey,None,None,None,None,110,150,German
3442,LUTTEN,Audi,A3,Audi-A3,15,1.8 TFSI Attraction 2x S-line Sportback,2007,6950,159000,2007-11-01,Gasoline,Privateseller,Other,Used,Front,5,4,None,None,NaN,2007-11-01,NaT,2022-06